In [1]:
# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
# This will automatically reload src/dtree.py every time you make changes and save the file
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

In [3]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [4]:
df = pd.read_csv('./test.csv')

In [5]:
d1 = df['original_reg_date'].isnull()
df['age'] = df['original_reg_date'].copy()
df['age'][d1] = df['reg_date'][d1]
d1 = df['age'].isnull()
df['age'][d1] = df['manufactured'][d1]

def extract_year(date):
    date = str(date)
    if len(date) != 4:
        if '.' in date and len(date) == 6:
            year = date[:-2]
        else:
            l = date.split('-')
            year = l[-1]
    return int(year)
df['age'] = 2021 - df['age'].map(extract_year)

G:\softwares\envs\4277lab\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
G:\softwares\envs\4277lab\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [6]:
# take out outliers
print(df['age'].isna().sum())
print(df['age'].median())

0
5.0


In [7]:
df['age'] = df['age'].fillna(5)

In [9]:
# take out outliers acccording to scatter plot
# df = df[~((df['power'] > 200) & (df['engine_cap'] < 1000))]

df_valid = df.dropna(subset = ['power', 'engine_cap'])
powers = df_valid['power'].to_numpy()
caps = df_valid['engine_cap'].to_numpy()

# Find k data points with engine cap most similar to the power of x
# Calculate the median power of those k data points
# Set power of x to the calculated median
# Set k as the number of nearest data points we want to consider
k = 500

# Loop over each data point in our dataset
for idx, row in df.iterrows():
    if idx % 10000 == 0:
        print(idx)

    p, e = row['power'], row['engine_cap']
    
    if type(row["power"]) == float and pd.isna(row["power"]):
        diff = np.abs(caps - e)

        indices = np.argsort(diff)[:k]

        k_powers = powers[indices]

        avg = np.average(k_powers)

        # Set the weight value of the current row to the median
        df.at[idx, 'power'] = avg

0


In [12]:
print(df['mileage'].isnull().sum())
df['mileage'] = df['mileage'].fillna(df['mileage'].median())

1144


In [13]:
# fill omv, get data with missing omv and dereg_value, fill in the omv
is_ = df[df.omv.isnull() & df.dereg_value.isnull()].index.tolist()
df['omv'] = df['omv'].fillna(27055.5)
print(len(df[df.omv.isnull() & df.dereg_value.isnull()]))

0


In [14]:
# coe: fill in missing value for coe according to: age and power
fill_arf = df[df.arf.isnull() & df.dereg_value.isnull()]['arf'].index.tolist()
for idx in fill_arf:
    omv = df.at[idx, 'omv']
    if omv < 20000:
        df.at[idx, 'arf'] = omv
    elif omv >= 20000 and omv <= 50000:
        df.at[idx, 'arf'] = (omv - 20000) * 1.40 + 20000
    else:
        df.at[idx, 'arf'] =  (omv - 50000) * 1.80 + 62000
print(len(df[df.arf.isnull() & df.dereg_value.isnull()]))
# df = df[~(df.arf.isnull() & df.dereg_value.isnull())] # remove values whose dereg_value cannot be calculated

0


In [15]:
df['age_in_month'] = df['age'] * 12

In [16]:
# recalculate reg date: total months
def to_months(x):
    mapper = {}
    all_m = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
    for i in range(12):
        mapper[all_m[i]] = i + 1
    if len(str(x)) < 11:
        return int(x)
    s = x.split('-')
    year = 2021 - int(s[-1])
    month = mapper[s[-2]]
    return year * 12 + month

df[~df.reg_date.isnull()]['age_in_month'] = df[~df.reg_date.isnull()]['reg_date'].map(to_months)

G:\softwares\envs\4277lab\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [17]:
# for date missing both coe and dereg_value, record it's month engine and power
coe_to_search = []
to_search = df[df.coe.isnull() & df.dereg_value.isnull()]

for idx, row in to_search.iterrows():
    # category -> A / B
    power = row["power"]
    engine_cap = row["engine_cap"]
    cat = 2
    month = row["age_in_month"]
    if engine_cap <= 1600 and power <= 97:
        cat = 1
    coe_to_search.append((cat, month))

coe_to_search = list(set(coe_to_search))
print(len(coe_to_search))

11


In [18]:
t_to_coe = {}

for t in coe_to_search:
    t_to_coe[str(t)] = []

remains = df[~df.coe.isnull()]

for idx, row in remains.iterrows():
    if idx % 10000 == 0:
        print(idx)

    power = row["power"]
    engine_cap = row["engine_cap"]
    cat = 2
    month = row["age_in_month"]
    if engine_cap <= 1600 and power <= 97:
        cat = 1
    
    t = (cat, month)
    
    coe = row["coe"]
    if t in coe_to_search:
        t_to_coe[str(t)].append(coe)
        
c = 0
for idx, row in df.iterrows():
    power = row["power"]
    engine_cap = row["engine_cap"]
    cat = 2
    month = row["age_in_month"]
    if engine_cap <= 1600 and power <= 97:
        cat = 1
    t = str((cat, month))

    if type(row["coe"]) == float and pd.isna(row["coe"]):
        l = t_to_coe[t]
        if len(l) > 0:
            avg = sum(l) / len(l)
            df.at[idx, 'coe'] = avg
            c += 1
print(c)

245


In [19]:
print(df.shape)

(5000, 31)


In [20]:
print(len(df[(df.coe.isnull() & df.dereg_value.isnull())]))
print(np.average(df['coe'].dropna()))

0
41969.19715303798


In [21]:
df['coe'] = df['coe'].fillna(42632)

In [22]:
print(df["road_tax"].median())
# df = df.fillna()
# df["road_tax"] = df["road_tax"].map(clean_tax)

976.0


In [23]:
df["road_tax"] = df["road_tax"].fillna(974)

In [24]:
import math
for idx, row in df.iterrows():
    if type(row["dereg_value"]) == float and pd.isna(row["dereg_value"]):
        max_age = 10 * 12
        age = row["age_in_month"]
        time_left = max_age - age
        coe = row["coe"]
        omv = row["omv"]
        arf = row["arf"]
        parf_ratio = 0
        if age <  5 * 12:
            parf_ratio = 0.75
        elif age >= 5 * 12 and age < 10 * 12:
            parf_ratio = 0.75 - math.ceil((age - 5 * 12) / 12) * 0.05 
        coe_rebate = coe * time_left / 120
        dereg_value = parf_ratio * arf + coe_rebate
        if dereg_value <= 0:
            dereg_value = 0
        df.at[idx, "dereg_value"] = dereg_value

In [25]:
# compare dereg value calculated
diff = []
print(df['arf'].isnull().values.any())
for idx, row in df.iterrows():
    if not(type(row["dereg_value"]) == float and pd.isna(row["dereg_value"])) and \
    not(type(row["arf"]) == float and pd.isna(row["arf"])):
        real = row["dereg_value"]
        max_age = 10 * 12
        age = row["age_in_month"]
        time_left = max_age - age
        coe = row["coe"]
        omv = row["omv"]
        arf = row["arf"]
        parf_ratio = 0
        if age <  5 * 12:
            parf_ratio = 0.75
        elif age >= 5 * 12 and age < 10 * 12:
            parf_ratio = 0.75 - math.ceil((age - 5 * 12) / 12) * 0.05 
        coe_rebate = coe * time_left / 120
        calculated = parf_ratio * arf + coe_rebate
        if age <= 120:
            print(real, calculated, coe, omv, arf, age)
            diff.append(abs(real - calculated))
        if idx == 10:
            break
print(np.average(np.array(diff)))

True
44938.24878048781 44938.24878048781 69915.48780487805 68994.0 68994.0 108
102072.0 105258.6 53106.0 69922.0 97860.0 48
73330.0 74332.35 47501.0 49363.0 61109.0 48
34555.0 35586.9 46489.0 20184.0 10258.0 48
30305.0 36547.4 78000.0 52268.0 52268.0 108
17890.0 19811.5 26000.0 14082.0 9082.0 60
22384.0 25074.6 64209.0 27420.0 20388.0 96
16699.0 18844.85 22237.0 28136.0 1407.0 24
54051.0 57585.25 47000.0 38176.0 45447.0 60
26357.0 26834.85 28589.0 29445.0 1473.0 12
2223.330000000001


In [26]:
avg = np.average(df['depreciation'].dropna())

In [27]:
df = df.fillna(avg)

In [28]:
# drop irrelevant
df = df.drop(columns = ['curb_weight', 'coe', 'omv', 'arf', 'original_reg_date','reg_date', 'age_in_month', 'opc_scheme', 'listing_id', 'indicative_price', 'age', 'mileage', 'manufactured', 'engine_cap', 'no_of_owners', 'lifespan', 'eco_category', 'accessories'])

In [29]:
print(df.shape)

(5000, 21)


In [30]:
df.to_csv('test_num_cleaned.csv')  

In [31]:
print(df.iloc[0])

title                       BMW X6 xDrive35i Sunroof (New 10-yr COE)
make                                                             bmw
model                                                             x6
description        owner consignment unit, viewing strictly by ap...
manufactured                                                  2012.0
type_of_vehicle                                                  suv
category                                                     coe car
transmission                                                    auto
power                                                        240.005
fuel_type                                               14432.730074
engine_cap                                                   1773.03
no_of_owners                                                     3.0
depreciation                                                 14870.0
road_tax                                                      2362.0
dereg_value                       